<a href="https://colab.research.google.com/github/kobrue02/BENALI/blob/main/notebooks/llama_responses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Initialize model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.3-70B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.3-70B-Instruct").to("cuda")

def get_l1_prediction(text, l1s):
    labels = "\n- ".join(l1s)
    prompt = """Consider the following text which was written by a non-native speaker:
    `{text}`
    \nWhat do you think is their native language?
    Choose one of the following:
    {labels}
    Return only the answer.
    The native language is: """.format(text=text, labels=l1s)

    messages = [
        {"role": "user", "content": prompt},
    ]

    # Apply chat template and tokenize
    inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt",
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        temperature=0.1,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode only the new tokens (response part)
    response = tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    print(f"Raw response: {response}")

    # Parse the response
    if "The native language is: " in response:
        result = response.split("The native language is: ")[1].strip()
        # Clean up common issues
        result = result.split('\n')[0].split('.')[0].strip()
        return result
    else:
        # Fallback parsing - look for any of the labels in the response
        response_lower = response.lower()
        for label in l1s:
            if label.lower() in response_lower:
                return label
        return "Unknown"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/879 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/59.6k [00:00<?, ?B/s]

...

model-00027-of-00030.safetensors:   0%|          | 0.00/4.66G [00:00<?, ?B/s]

model-00028-of-00030.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00029-of-00030.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00030-of-00030.safetensors:   0%|          | 0.00/2.10G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [28]:
import pandas as pd

test_df = pd.read_csv("nli_test.csv")
test_df.head()

,sentence,l1
0,It would be moree accurate to say Athens (3 mi...,Greece
1,#编号：540\n#篇名：兄弟射雁\n#字数／字符数：411/411\n#类型：习作\n#姓...,Vietnam
2,"Not that they are poor, they just don't give m...",Portugal
3,Red in the corners with white surrounding them,Norway
4,>Wenn es stets zu Schutz und Trutze,Germany


In [61]:
test_sample = test_df.iloc[0]
test_sample

,0
sentence,It would be moree accurate to say Athens (3 mi...
l1,Greece


In [73]:
model_response = get_l1_prediction(test_sample["sentence"], test_df["l1"].unique())
model_response

[{'generated_text': ' Consider the following text which was written by a non-native speaker:\n    `It would be moree accurate to say Athens (3 million), Thessaloniki (800,000) and Patras (210,000)`\n    Your task is to determine the native language (L1) of the author.\n    What is their most likely native language? Choose from:\n    1. Greece\n2. Vietnam\n3. Portugal\n4. Norway\n5. Germany\n6. Slovenia\n7. Russia\n8. Sweden\n9. Japan\n10. Finland\n11. Ireland\n12. Belgium\n13. Turkey\n14. Denmark\n15. Bosnia and Herzegovina\n16. Italy\n17. Poland\n18. Czechia\n19. Romania\n20. North Macedonia\n21. United Kingdom\n22. Netherlands\n23. Hungary\n24. France\n25. Iceland\n26. Montenegro\n27. Serbia\n28. Bulgaria\n29. United States\n30. Argentina\n31. Austria\n32. Switzerland\n33. Kazakhstan\n34. Greenland\n35. Spain\n36. Madagascar\n37. Canada\n38. England\n39. Croatia\n40. Ukraine\n41. Lithuania\n42. Indonesia\n43. Cyprus\n44. Israel\n45. Wales\n46. China\n47. Scotland\n48. Estonia\n49. Be

'Japan'

In [75]:
# on the full test set
from tqdm import tqdm
from transformers import logging as transformers_logging
import warnings

transformers_logging.set_verbosity_error()
warnings.filterwarnings("ignore")
tqdm.pandas()
labels = test_df["l1"].unique()
test_sample = test_df.sample(1000)
test_sample["l1_pred"] = test_sample.progress_apply(lambda x: get_l1_prediction(x["sentence"], labels), axis=1)

  0%|          | 3/1000 [00:09<45:07,  2.72s/it]  

[{'generated_text': " Consider the following text which was written by a non-native speaker:\n    `Too many things must change for that to happen`\n    Your task is to determine the native language (L1) of the author.\n    What is their most likely native language? Choose from:\n    1. Greece\n2. Vietnam\n3. Portugal\n4. Norway\n5. Germany\n6. Slovenia\n7. Russia\n8. Sweden\n9. Japan\n10. Finland\n11. Ireland\n12. Belgium\n13. Turkey\n14. Denmark\n15. Bosnia and Herzegovina\n16. Italy\n17. Poland\n18. Czechia\n19. Romania\n20. North Macedonia\n21. United Kingdom\n22. Netherlands\n23. Hungary\n24. France\n25. Iceland\n26. Montenegro\n27. Serbia\n28. Bulgaria\n29. United States\n30. Argentina\n31. Austria\n32. Switzerland\n33. Kazakhstan\n34. Greenland\n35. Spain\n36. Madagascar\n37. Canada\n38. England\n39. Croatia\n40. Ukraine\n41. Lithuania\n42. Indonesia\n43. Cyprus\n44. Israel\n45. Wales\n46. China\n47. Scotland\n48. Estonia\n49. Belarus\n50. Slovakia\n51. Iran\n52. South Korea\n53.

  1%|          | 6/1000 [00:18<45:17,  2.73s/it]  

[{'generated_text': ' Consider the following text which was written by a non-native speaker:\n    `And the recent Samuel L. Jackson movie Big Game (2014) was funnily set in Finland, but was actually filmed in Germany and in the Alps(!)`\n    Your task is to determine the native language (L1) of the author.\n    What is their most likely native language? Choose from:\n    1. Greece\n2. Vietnam\n3. Portugal\n4. Norway\n5. Germany\n6. Slovenia\n7. Russia\n8. Sweden\n9. Japan\n10. Finland\n11. Ireland\n12. Belgium\n13. Turkey\n14. Denmark\n15. Bosnia and Herzegovina\n16. Italy\n17. Poland\n18. Czechia\n19. Romania\n20. North Macedonia\n21. United Kingdom\n22. Netherlands\n23. Hungary\n24. France\n25. Iceland\n26. Montenegro\n27. Serbia\n28. Bulgaria\n29. United States\n30. Argentina\n31. Austria\n32. Switzerland\n33. Kazakhstan\n34. Greenland\n35. Spain\n36. Madagascar\n37. Canada\n38. England\n39. Croatia\n40. Ukraine\n41. Lithuania\n42. Indonesia\n43. Cyprus\n44. Israel\n45. Wales\n46. C

  1%|          | 7/1000 [00:18<32:56,  1.99s/it]

...

[{'generated_text': " Consider the following text which was written by a non-native speaker:\n    `We set them up strategically to kill our targets and it's mostly effective`\n    Your task is to determine the native language (L1) of the author.\n    What is their most likely native language? Choose from:\n    1. Greece\n2. Vietnam\n3. Portugal\n4. Norway\n5. Germany\n6. Slovenia\n7. Russia\n8. Sweden\n9. Japan\n10. Finland\n11. Ireland\n12. Belgium\n13. Turkey\n14. Denmark\n15. Bosnia and Herzegovina\n16. Italy\n17. Poland\n18. Czechia\n19. Romania\n20. North Macedonia\n21. United Kingdom\n22. Netherlands\n23. Hungary\n24. France\n25. Iceland\n26. Montenegro\n27. Serbia\n28. Bulgaria\n29. United States\n30. Argentina\n31. Austria\n32. Switzerland\n33. Kazakhstan\n34. Greenland\n35. Spain\n36. Madagascar\n37. Canada\n38. England\n39. Croatia\n40. Ukraine\n41. Lithuania\n42. Indonesia\n43. Cyprus\n44. Israel\n45. Wales\n46. China\n47. Scotland\n48. Estonia\n49. Belarus\n50. Slovakia\n51

  1%|▏         | 13/1000 [00:29<37:14,  2.26s/it]

[{'generated_text': ' Consider the following text which was written by a non-native speaker:\n    `Imagine that: Nazis find you too inhuman`\n    Your task is to determine the native language (L1) of the author.\n    What is their most likely native language? Choose from:\n    1. Greece\n2. Vietnam\n3. Portugal\n4. Norway\n5. Germany\n6. Slovenia\n7. Russia\n8. Sweden\n9. Japan\n10. Finland\n11. Ireland\n12. Belgium\n13. Turkey\n14. Denmark\n15. Bosnia and Herzegovina\n16. Italy\n17. Poland\n18. Czechia\n19. Romania\n20. North Macedonia\n21. United Kingdom\n22. Netherlands\n23. Hungary\n24. France\n25. Iceland\n26. Montenegro\n27. Serbia\n28. Bulgaria\n29. United States\n30. Argentina\n31. Austria\n32. Switzerland\n33. Kazakhstan\n34. Greenland\n35. Spain\n36. Madagascar\n37. Canada\n38. England\n39. Croatia\n40. Ukraine\n41. Lithuania\n42. Indonesia\n43. Cyprus\n44. Israel\n45. Wales\n46. China\n47. Scotland\n48. Estonia\n49. Belarus\n50. Slovakia\n51. Iran\n52. South Korea\n53. Latvi

  1%|▏         | 13/1000 [00:34<44:07,  2.68s/it]


KeyboardInterrupt: 

In [74]:
import torch
torch.cuda.empty_cache()

# also run garbage collection
import gc
gc.collect()

96

In [17]:
test_sample.to_csv("llama_predictions_sample.csv", index=False)

In [18]:
# classification report on the l1 predictions
from sklearn.metrics import classification_report

print(classification_report(test_sample["l1"], test_sample["l1_pred"]))